## Preface

Upon comparing San Diego's COVID trend to other major cities, I found that the new daily cases of riverside drastically dropped down at 7/17/20, which is the date that the Trump administration issued a change in COVID data management from the CDC to the HHSA. There has been much speculation about the transparency of the HHSA's release of COVID data, and the outlier of riverside inspired the initiative to examine if there are other major counties in the US that has a drastic drop in COVID data around the same time. 

## Data Import

In [1]:
import pandas as pd 
import numpy as np 
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats
from itertools import chain 

In [2]:
df_covid = pd.read_csv('Datasets/COVID-19_Statistics_by_Jurisdiction.csv')
## https://sdgis-sandag.opendata.arcgis.com/datasets/covid-19-statistics-by-jurisdiction?geometry=-120.348%2C32.566%2C-115.965%2C33.372
df_population = pd.read_csv('Datasets/sub-est2019_6.csv', encoding = 'latin-1')
ca_population = pd.read_csv('Datasets/sub-est2019_6.csv', encoding = 'latin-1')
ca_covid = pd.read_csv('Datasets/statewide_cases.csv')

## Data Cleaning